In [3]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import torch

inst = 'VGNSL_split/instances_val2014.json'
coco=COCO(inst)

Using cache found in /Users/sisi/.cache/torch/hub/pytorch_vision_v0.6.0


loading annotations into memory...
Done (t=4.35s)
creating index...
index created!


In [64]:
import urllib
from PIL import Image
from torchvision import transforms
from torchvision import models

In [66]:
# use ResNet 101
model = models.resnet101(pretrained=True)
layer = model._modules.get('avgpool')

for param in model.parameters():
    param.requires_grad = False
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [119]:
# get image from coco using image ids
imgIds = coco.getImgIds(imgIds = [184613])
# print(imgIds)
img = coco.loadImgs(imgIds)[0]
url = img['coco_url']
filename = img['file_name']
# download image using PIL
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename) 
input_image = Image.open(filename)

# preprocess image 
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create minibatch 
# create hook
outputs= []

def hook(module, input, output):
    outputs.append(input)
    
h = layer.register_forward_hook(hook)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
    
with torch.no_grad():
    out = model(input_batch)
# h.remove()
# print(outputs[0].shape)

a = torch.squeeze(outputs[0]).numpy()
a

array([1.9807956 , 0.51592296, 1.3015637 , ..., 0.37959543, 0.58813417,
       0.43346038], dtype=float32)

In [120]:
vgnsl_im = np.load('VGNSL_split/dev_ims.npy')
vgnsl_img = vgnsl_im[0]
vgnsl_img

array([0.97975415, 1.6569574 , 1.7241993 , ..., 0.15878499, 1.9806446 ,
       2.1665163 ], dtype=float32)

In [121]:
a == vgnsl_img

array([False, False, False, ..., False, False, False])

In [5]:
img.keys()

dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])